In [204]:
import pandas as pd
import numpy as np

In [205]:
import os
path = os.getcwd() #get working directory

pd_file_names = [] #empty array for filenames
tms_file_names = [] 
pa_file_names = []

#append all file names within folders into list
for (dirpath, dirnames, filenames) in os.walk(path + '/player_data/'):
    pd_file_names.extend(filenames)
    break

for (dirpath, dirnames, filenames) in os.walk(path + '/team_misc_data/'):
    tms_file_names.extend(filenames)
    break

for (dirpath, dirnames, filenames) in os.walk(path + '/player_advanced_data/'):
    pa_file_names.extend(filenames)
    break
    
#drop '.DS_Store'
pd_file_names.remove('.DS_Store')
tms_file_names.remove('.DS_Store')
pa_file_names.remove('.DS_Store')

In [206]:
pd_list = [] #empty list for data storage
tms_list = []
pa_list = []

#read in player data
for i in pd_file_names:
    data = pd.read_csv(path + '/player_data/' + i, sep=',', quoting=3)
    pd_list.append(data)
#read in team data
for i in tms_file_names:
    data = pd.read_csv(path + '/team_misc_data/' + i, sep=',', quoting=3)
    tms_list.append(data)
#read in player advanced data
for i in pa_file_names:
    data = pd.read_csv(path + '/player_advanced_data/' + i, sep=',', quoting=3)
    pa_list.append(data)

In [207]:
#create list of seasons to append to each data set within list
season_list = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
              2012, 2013, 2014, 2015, 2016, 2017]

#empty lists
pd_list_new = []
tms_list_new = []
pa_list_new = []

#for each data set & season
for h,i,j,k in zip(pa_list, pd_list, tms_list, season_list):
    #create data frame with season repeating the same length as player/team data
    season_pa = pd.DataFrame(np.repeat(k, len(h)))
    season_pd = pd.DataFrame(np.repeat(k, len(i)))
    season_tms = pd.DataFrame(np.repeat(k, len(j)))
    #name column
    season_pd.columns = ['season']; season_tms.columns = ['season']
    #concatentate column and return data to list
    data_pa = pd.concat([h, season_pa], axis=1)
    data_pd = pd.concat([i, season_pd], axis=1)
    data_tms = pd.concat([j, season_tms], axis=1)
    pa_list_new.append(data_pa)
    pd_list_new.append(data_pd)
    tms_list_new.append(data_tms)

In [208]:
#Player data cleaning
pd_data = pd.concat(pd_list_new, axis=0) #concatenate all data into single data set

#Check which colums have too many NA values
pd_data.isnull().sum()

#Based on manual analysis (to understand the data)..
drop_columns = ['"Rk']

#Drop the columns
pd_data = pd_data.drop(pd_data.loc[:,drop_columns].columns, axis=1)

#Rename column + remove quote from values + convert to float
pd_data.rename(columns={'PS/G"': 'pts'}, inplace=True)
pd_data['pts'] = pd_data['pts'].str.replace('"', "")
pd_data['pts'] = pd_data['pts'].astype(float)

#remove player code from player name
players = pd_data['Player'].str.partition("\\")[0]
pd_data['Player'] = players

#lower the column names
pd_columns = pd_data.columns.str.lower()
pd_data.columns = pd_columns

#reset index
pd_data.reset_index(drop=True, inplace=True)

In [209]:
#Player advanced data cleansing
pa_data = pd.concat(pa_list_new, axis=0) #concatenate all data into single data set

#Check which colums have too many NA values
pa_data.isnull().sum()

#Based on manual analysis (to understand the data)..
drop_columns = ['"Rk', 'Pos', 'Age', 'G', 'MP', 'Unnamed: 19', 'Unnamed: 24']

#Drop the columns
pa_data = pa_data.drop(pa_data.loc[:,drop_columns].columns, axis=1)

#Rename column + remove quote from values + convert to float
pa_data.rename(columns={'VORP"': 'vorp', 0: 'season'}, inplace=True)
pa_data['vorp'] = pa_data['vorp'].str.replace('"', "")
pa_data['vorp'] = pa_data['vorp'].astype(float)

#remove player code from player name
players = pa_data['Player'].str.partition("\\")[0]
pa_data['Player'] = players

#lower the column names
pa_columns = pa_data.columns.str.lower()
pa_data.columns = pa_columns

#reset index
pa_data.reset_index(drop=True, inplace=True)

In [210]:
#Team data cleansing
tms_data = pd.concat(tms_list_new, axis=0) #concatenate all data into single data set

#Which columns to remove based on the statistic definition?
# remove pythagorean wins/losses, will be hard to reproduce within player stats
# remove margin of victory, can't repro with plaer stats
# remove sos, not applicable
# remove srs, not applicable
# remove pace, not going to able to reproduce
# remove arena & attendance
drop_columns= ['"Rk', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'Pace', 'Arena', 'Attendance"']

#Drop the columns
tms_data = tms_data.drop(tms_data.loc[:,drop_columns].columns, axis=1)

#Rename the redundant columns related to defensive efficiency
tms_data.rename(columns={'eFG%.1':'eFG%_def', 'TOV%.1':'TOV%_def', 'FT/FGA.1': 'FT/FGA_def'}, inplace=True)

#lower the column names
tms_columns = tms_data.columns.str.lower()
tms_data.columns = tms_columns

#Remove season averages from data, found via exploratory analysis
season_average_data = tms_data[tms_data.isnull().any(axis=1)] #extract season averages
tms_data = tms_data[-tms_data.isnull().any(axis=1)] #remove from tms_data

#Handle asterix indicating playoffs within team name
partition = tms_data['team'].str.partition("*")
partition[1].replace(["*", ""], ["map", "mip"], inplace=True) #{map: made playofs, mip: missed playoffs}

playoff_data = partition.loc[:,0:1] #create new data about whether team made playoffs
playoff_data = pd.concat([playoff_data, tms_data['season']], axis=1) #add season for merge purpose
playoff_data.rename(columns={'0':'team', '1':'playoff_status'}, inplace=True) #clean columns

tms_data['team'] = partition[0] #return cleaned names to tms data

#adding winnig percentage to tms_data
tms_data['wp'] = tms_data['w'] / (tms_data['l'] + tms_data['w'])

#reset index
tms_data.reset_index(drop=True, inplace=True)
season_average_data.reset_index(drop=True, inplace=True)
playoff_data.reset_index(drop=True, inplace=True)

In [211]:
#Clean & handle the discrepencies in team names across data sets
tms_data['team'].unique() #reviewed team names
pd_data['tm'].unique() #review abbreviations
pa_data['tm'].unique()

#Drop pd_data with 'TOT' abbreviation - indicates player was on multiple teams
pd_data = pd_data.loc[pd_data['tm'] != 'TOT']
pa_data = pa_data.loc[pa_data['tm'] != 'TOT']
#Remove tms_data for New Orleans/OKH - no matching abbreviation ... NEED TO DO TO PLAYOFFS DATA
tms_data = tms_data.loc[tms_data['team'] != 'New Orleans/Oklahoma City Hornets']

#import abbreviation data built from seeing unique values between data sets
team_abbr_map = pd.read_csv(path + '/team_abbr_mapping.csv')
team_abbr_map.columns = ['team_acronym', 'team_name']

#merge standardized team name / abbreviation into data sets
pd_data = pd_data.merge(team_abbr_map, left_on = 'tm', right_on = 'team_acronym', how='left')
pa_data = pa_data.merge(team_abbr_map, left_on = 'tm', right_on = 'team_acronym', how='left')

pd_data['team_acronym'] = pd_data['team_acronym'].str.lower()
pa_data['team_acronym'] = pa_data['team_acronym'].str.lower()

pd_data['team_name'] = pd_data['team_name'].str.lower()
pa_data['team_name'] = pa_data['team_name'].str.lower()

pd_data['team_name'] = pd_data['team_name'].str.replace(" ", "_")
pa_data['team_name'] = pa_data['team_name'].str.replace(" ", "_")

del pd_data['tm'] 
del pa_data['tm']
#del pd_data['index']
#del pa_data['index']

#team data is already consistent with team names
tms_data.rename(columns={'team': 'team_name'}, inplace=True)
tms_data['team_name'] = tms_data['team_name'].str.lower()
tms_data['team_name'] = tms_data['team_name'].str.replace(" ", "_")


In [212]:
#Merge the player data together
pd_pa_data = pd_data.merge(pa_data, on=['player', 'team_name', 'season'], how='left')
pd_pa_data.columns

#Drop columns
pd_pa_data = pd_pa_data.drop('team_acronym_y', axis=1)
pd_pa_data.rename(columns={'team_acronym_x':'team_acronym'}, inplace=True)

In [213]:
#Next steps:
# 1.create function that will aggregate team stats by the players input...
# 2. then use this function to aggregate the player stats per team per year for each season...
# 3. and use this data set to train the model against wins

# 1.create function that will aggregate team stats by the players input...
def aggregatePlayerStats(data, player_list):
    team_data = data[data['player'].isin(player_list)] #subset data in the player list
    sum_data = team_data[['fg', 'fga', '3p', '3pa', '2p', '2pa', 'ft', 'fta',
                       'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'mp']].sum()
    avg_data = team_data[['2p%', '3p%', 'fg%', 'ft%', 'efg%', 'age']].mean()
    final = pd.DataFrame(pd.concat([sum_data, avg_data], axis=0)).transpose()
    return final

In [214]:
# 1.create function that will aggregate team stats by the players input...
# 2. then use this function to aggregate the player stats per team per year for each season...
# 3. and use this data set to train the model against wins

# 1.create function that will aggregate team stats by the players input...
def aggregatePlayerStatsAdvanced(data, player_list):
    team_data = data[data['player'].isin(player_list)] #subset data in the player list
    sum_data = team_data[['fg', 'fga', '3p', '3pa', '2p', '2pa', 'ft', 'fta',
                       'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'mp']].sum()
    avg_data = team_data[['2p%', '3p%', 'fg%', 'ft%', 'efg%', 'age', 'per', '3par',
                         'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
                         'usg%', 'ows', 'dws', 'ws', 'ws/48', 'obpm', 'dbpm', 'bpm', 'vorp']].mean()
    final = pd.DataFrame(pd.concat([sum_data, avg_data], axis=0)).transpose()
    return final

In [215]:
# 2. then use this function to aggregate the player stats per team per year for each season...
historical_team_data = []
import itertools
for i, j in itertools.product(pd_data['team_name'].unique(), pd_data['season'].unique()):
    #subset player data
    team_year_data = pd_data.loc[(pd_data.team_name == i) & (pd_data.season == j)]    
    #aggreagete team stats 
    agg_stats = aggregatePlayerStats(team_year_data, team_year_data['player'].unique())
    #include win losses for the team that year
    agg_stats = pd.concat([pd.DataFrame(tms_data.loc[(tms_data.team_name == i) & 
                    (tms_data.season == j), ['w', 'l', 'wp']]).reset_index(drop=True),
                       agg_stats], axis=1)
    final = pd.concat([pd.DataFrame({'team_name': [i], 'season': [j]}), agg_stats], axis=1)
    historical_team_data.append(final)  
                      
historical_team_data = pd.concat(historical_team_data, axis=0) #concatenate all data into single data set


In [216]:
# 2. then use this function to aggregate the player stats per team per year for each season...
historical_team_data_top_5 = []
import itertools
#for each combination of team & season
for i, j in itertools.product(pd_data['team_name'].unique(), pd_data['season'].unique()):
    #subset player data
    team_year_data = pd_data.loc[(pd_data.team_name == i) & (pd_data.season == j)]
    #identify top 5 players by minutes played
    team_year_data_top_5 = team_year_data.sort_values(by='mp', axis=0, ascending=False)[0:4]
    #aggregate team stats
    agg_stats = aggregatePlayerStats(team_year_data, team_year_data_top_5['player'].unique())
    #include win losses for the team that year
    agg_stats = pd.concat([pd.DataFrame(tms_data.loc[(tms_data.team_name == i) & 
                    (tms_data.season == j), ['w', 'l', 'wp']]).reset_index(drop=True),
                       agg_stats], axis=1)
    #combine all stats
    final = pd.concat([pd.DataFrame({'team_name': [i], 'season': [j]}), agg_stats], axis=1)
    historical_team_data_top_5.append(final)
                      
historical_team_data_top_5 = pd.concat(historical_team_data_top_5, axis=0) #concatenate all data into single data set

In [217]:
#this will be top 5 based on the # of games started... generally this is the squad
historical_team_data_top_5_gs = []
import itertools
#for each combination of team & season
for i, j in itertools.product(pd_data['team_name'].unique(), pd_data['season'].unique()):
    #subset player data
    team_year_data = pd_data.loc[(pd_data.team_name == i) & (pd_data.season == j)]
    #identify top 5 players by minutes played
    team_year_data_top_5 = team_year_data.sort_values(by='gs', axis=0, ascending=False)[0:4]
    #aggregate team stats
    agg_stats = aggregatePlayerStats(team_year_data, team_year_data_top_5['player'].unique())
    #include win losses for the team that year
    agg_stats = pd.concat([pd.DataFrame(tms_data.loc[(tms_data.team_name == i) & 
                    (tms_data.season == j), ['w', 'l', 'wp']]).reset_index(drop=True),
                       agg_stats], axis=1)
    #combine all stats
    final = pd.concat([pd.DataFrame({'team_name': [i], 'season': [j]}), agg_stats], axis=1)
    historical_team_data_top_5_gs.append(final)
                      
historical_team_data_top_5_gs = pd.concat(historical_team_data_top_5_gs, axis=0) #concatenate all data into single data set

In [218]:
# 2. then use this function to aggregate the player stats per team per year for each season...
historical_team_data_top_5_advanced = []
import itertools
#for each combination of team & season
for i, j in itertools.product(pd_pa_data['team_name'].unique(), pd_pa_data['season'].unique()):
    #subset player data
    team_year_data = pd_pa_data.loc[(pd_pa_data.team_name == i) & (pd_pa_data.season == j)]
    #identify top 5 players by minutes played
    team_year_data_top_5 = team_year_data.sort_values(by='mp', axis=0, ascending=False)[0:4]
    #aggregate team stats
    agg_stats = aggregatePlayerStatsAdvanced(team_year_data, team_year_data_top_5['player'].unique())
    #include win losses for the team that year
    agg_stats = pd.concat([pd.DataFrame(tms_data.loc[(tms_data.team_name == i) & 
                    (tms_data.season == j), ['w', 'l', 'wp']]).reset_index(drop=True),
                       agg_stats], axis=1)
    #combine all stats
    final = pd.concat([pd.DataFrame({'team_name': [i], 'season': [j]}), agg_stats], axis=1)
    historical_team_data_top_5_advanced.append(final)
                      
historical_team_data_top_5_advanced = pd.concat(historical_team_data_top_5_advanced, axis=0)

In [220]:
#remove records without wins data, team didn't exist
#something wrong with 2015 data, will investigate at another time
historical_team_data = historical_team_data.loc[(historical_team_data['w'].notnull())]
historical_team_data_top_5 = historical_team_data_top_5.loc[(historical_team_data_top_5['w'].notnull())]
historical_team_data_top_5_gs = historical_team_data_top_5_gs.loc[(historical_team_data_top_5_gs['w'].notnull())]
historical_team_data_top_5_advanced = historical_team_data_top_5_advanced.loc[(
    historical_team_data_top_5_advanced['w'].notnull())]

In [221]:
#create a historical player data set that only includes 1 record per player per season, based on games played on team
player_data_sr = pd.DataFrame()
for i in pd_data['season'].unique():
    season_data = pd_data[pd_data['season']==i]
    season_data_sr = season_data.sort_values('g', ascending=False).groupby('player', as_index=False).first()
    player_data_sr = player_data_sr.append(season_data_sr)
    
#create a historical player data set that only includes 1 record per player per season, based on games played on team
player_data_sr_advanced = pd.DataFrame()
for i in pd_pa_data['season'].unique():
    season_data = pd_pa_data[pd_pa_data['season']==i]
    season_data_sr = season_data.sort_values('g', ascending=False).groupby('player', as_index=False).first()
    player_data_sr_advanced = player_data_sr_advanced.append(season_data_sr)

In [222]:
##get the average stats per team for top 5 players

team_data = historical_team_data_top_5
#relevant data for z scores are...
team_data.columns #fg%, fga, 3p%, 3pa, 2pa, 2p%, fta, ft%, ord, drb, trb, ast, stl, blk, tov, pf, pts, efg%
#removing age because it directionality of stat is unclear (older versus younger... probably middle is better)
team_data = team_data[['season','fg%', 'fga', '3p%', '3pa', '2p%', '2pa', 'ft%', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl',
                      'blk', 'tov', 'pf', 'pts', 'efg%', 'age']]
average_stats = pd.DataFrame()
seasons = team_data['season'].unique()

temp_data = team_data.drop('season', axis=1)
for s in seasons:
    season_data = temp_data[team_data['season'] == s]
    season_statistics_top_5 = pd.DataFrame(columns = ['mean', 'sd'])
    season_statistics_top_5['mean'] = season_data.mean()
    season_statistics_top_5['sd'] = season_data.std()
    season_statistics_top_5 = season_statistics_top_5.transpose()
    season_statistics_top_5['season'] = s
    average_stats = average_stats.append(season_statistics_top_5)

In [223]:
##get the average stats per team for top 5 players based on the number of agmes started 

team_data = historical_team_data_top_5_gs
#relevant data for z scores are...
team_data.columns #fg%, fga, 3p%, 3pa, 2pa, 2p%, fta, ft%, ord, drb, trb, ast, stl, blk, tov, pf, pts, efg%
#removing age because it directionality of stat is unclear (older versus younger... probably middle is better)
team_data = team_data[['season','fg%', 'fga', '3p%', '3pa', '2p%', '2pa', 'ft%', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl',
                      'blk', 'tov', 'pf', 'pts', 'efg%', 'age']]
average_stats_gs = pd.DataFrame()
seasons = team_data['season'].unique()

temp_data = team_data.drop('season', axis=1)
for s in seasons:
    season_data = temp_data[team_data['season'] == s]
    season_statistics_top_5 = pd.DataFrame(columns = ['mean', 'sd'])
    season_statistics_top_5['mean'] = season_data.mean()
    season_statistics_top_5['sd'] = season_data.std()
    season_statistics_top_5 = season_statistics_top_5.transpose()
    season_statistics_top_5['season'] = s
    average_stats_gs = average_stats_gs.append(season_statistics_top_5)

In [224]:
##get the average stats per team for top 5 players based on the number of agmes started 

team_data = historical_team_data_top_5_advanced
#relevant data for z scores are...
team_data.columns #fg%, fga, 3p%, 3pa, 2pa, 2p%, fta, ft%, ord, drb, trb, ast, stl, blk, tov, pf, pts, efg%
team_data = team_data[['season','fg%', 'fga', '3p%', '3pa', '2p%', '2pa', 'ft%', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl',
                      'blk', 'tov', 'pf', 'pts', 'efg%', 'age', 'per', '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 
                       'stl%', 'blk%', 'tov%','usg%', 'ows', 'dws', 'ws', 'ws/48', 'obpm', 'dbpm', 'bpm', 'vorp']]
average_stats_advanced = pd.DataFrame()
seasons = team_data['season'].unique()

temp_data = team_data.drop('season', axis=1)
for s in seasons:
    season_data = temp_data[team_data['season'] == s]
    season_statistics_top_5 = pd.DataFrame(columns = ['mean', 'sd'])
    season_statistics_top_5['mean'] = season_data.mean()
    season_statistics_top_5['sd'] = season_data.std()
    season_statistics_top_5 = season_statistics_top_5.transpose()
    season_statistics_top_5['season'] = s
    average_stats_advanced = average_stats_advanced.append(season_statistics_top_5)

In [226]:
#write data sets to csv
historical_team_data.to_csv(path_or_buf=path + '/processed_data/historical_team_data.csv', index=False)
historical_team_data_top_5.to_csv(path_or_buf=path + '/processed_data/historical_team_data_top_5.csv', index=False)
historical_team_data_top_5_gs.to_csv(path_or_buf=path + '/processed_data/historical_team_data_top_5_gs.csv', index=False)
historical_team_data_top_5_advanced.to_csv(path_or_buf=path + '/processed_data/historical_team_data_top_5_advanced.csv', index=False)
pd_data.to_csv(path_or_buf=path + '/processed_data/player_data.csv', index=False)
pd_pa_data.to_csv(path_or_buf=path + '/processed_data/player_advanced_data.csv', index=False)
tms_data.to_csv(path_or_buf=path + '/processed_data/team_misc_stats_data.csv', index=False)
season_average_data.to_csv(path_or_buf=path + '/processed_data/season_average_data.csv', index=False)
playoff_data.to_csv(path_or_buf=path + '/processed_data/playoff_data.csv', index=False)
player_data_sr.to_csv(path + "/processed_data/player_data_single_record.csv", index=False)
player_data_sr_advanced.to_csv(path + '/processed_data/player_data_single_record_advanced.csv', index=False)
average_stats.to_csv(path + '/processed_data/season_average_top_5_data.csv', index=True)
average_stats_gs.to_csv(path + '/processed_data/season_average_top_5_gs_data.csv', index=True)
average_stats_advanced.to_csv(path + '/processed_data/season_average_top_5_advanced_data.csv', index=True)